In [1]:

def cycle_zigzag_folding(n: int) -> list[tuple[int, int]]:
    def get_main_direction(step_index: int) -> str:
        directions = ['r', 'd', 'l', 'u']  # right, down, left, up
        return directions[step_index % 4]

    def get_burst_directions(direction: str) -> list[str]:
        if direction == 'r':
            return ['u', 'd']
        elif direction == 'l':
            return ['d', 'u']
        elif direction == 'd':
            return ['r', 'l']
        elif direction == 'u':
            return ['l', 'r']
        return []

    def move(x: int, y: int, direction: str) -> tuple[int, int]:
        if direction == 'r':
            return x + 1, y
        elif direction == 'l':
            return x - 1, y
        elif direction == 'u':
            return x, y + 1
        elif direction == 'd':
            return x, y - 1
        return x, y

    # Initialize variables
    x, y = 0, 0
    path = [(x, y)]
    step_index = 0
    main_dir_width = 2
    continuous_main_steps = 0
    main_direction = get_main_direction(step_index)

    if n > 1:
        burst_counter = 0
        use_burst_direction = False
        x, y = move(x, y, 'd')
        path.append((x, y))

    for i in range(1, n - 1):
        if use_burst_direction:
            burst_directions = get_burst_directions(main_direction)
            burst_direction = burst_directions[burst_counter % 2]  # Alternate burst direction
            x, y = move(x, y, burst_direction)
            burst_counter += 1
        else:
            x, y = move(x, y, main_direction)
            continuous_main_steps += 1

        if (x, y) in path:
            print(f"Bug detected at step {i}: position {(x, y)} already in path.")
        path.append((x, y))
        use_burst_direction = not use_burst_direction

        # Adjust main direction and width after completing current main segment
        if continuous_main_steps >= main_dir_width:
            step_index += 1
            continuous_main_steps = 0
            burst_counter = 0
            main_direction = get_main_direction(step_index)
            use_burst_direction = not use_burst_direction
            if step_index % 2 == 0:
                main_dir_width += 2

    # Normalize coordinates to start from (0, 0)
    x_min = min(px for px, _ in path)
    y_min = min(py for _, py in path)
    normalized_path = [(px - x_min, py - y_min) for px, py in path]

    return normalized_path

In [2]:
def tikz_draw_zigzag(n: int) -> str:
    """
    Uses cycle_zigzag_folding(n) to create a TikZ line from 
    the returned coordinates.  Returns a string of TikZ commands.
    """
    coords = cycle_zigzag_folding(n)
    
    # Build the path:  (x1,y1) -- (x2,y2) -- ... -- (xk,yk)
    path = " -- ".join(f"({x},{y})" for (x,y) in coords)
    
    # Return something like:  \draw (x1,y1) -- (x2,y2) -- ... -- (xk,yk);
    return f"\\draw {path};"

In [3]:
tikz_draw_zigzag(92)

'\\draw (4,5) -- (4,4) -- (5,4) -- (5,5) -- (6,5) -- (6,4) -- (7,4) -- (7,3) -- (6,3) -- (6,2) -- (5,2) -- (5,3) -- (4,3) -- (4,2) -- (3,2) -- (3,3) -- (2,3) -- (2,4) -- (3,4) -- (3,5) -- (2,5) -- (2,6) -- (3,6) -- (3,7) -- (4,7) -- (4,6) -- (5,6) -- (5,7) -- (6,7) -- (6,6) -- (7,6) -- (7,7) -- (8,7) -- (8,6) -- (9,6) -- (9,5) -- (8,5) -- (8,4) -- (9,4) -- (9,3) -- (8,3) -- (8,2) -- (9,2) -- (9,1) -- (8,1) -- (8,0) -- (7,0) -- (7,1) -- (6,1) -- (6,0) -- (5,0) -- (5,1) -- (4,1) -- (4,0) -- (3,0) -- (3,1) -- (2,1) -- (2,0) -- (1,0) -- (1,1) -- (0,1) -- (0,2) -- (1,2) -- (1,3) -- (0,3) -- (0,4) -- (1,4) -- (1,5) -- (0,5) -- (0,6) -- (1,6) -- (1,7) -- (0,7) -- (0,8) -- (1,8) -- (1,9) -- (2,9) -- (2,8) -- (3,8) -- (3,9) -- (4,9) -- (4,8) -- (5,8) -- (5,9) -- (6,9) -- (6,8) -- (7,8) -- (7,9) -- (8,9) -- (8,8) -- (9,8) -- (9,9);'